In [17]:
# first, import the circuit benchmarks

# second, run transpilation on benchmarks for many iterations and save results
# want to show how much variance there is in the results
# NOTE important, use the same placement and routing before passed to transpiler so has a fair comparison

# third, use longest path to convert to fidelities
# we want to calculate expected fidelity improvement factor
# is fidelity improvement factor the same as the ratio between lengths (?)
# 

In [18]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR) # turn off for now

#  CONSTANTS
# repeat 10 times, only keep best
data_repitition = range(1)

# 1Q:2Q gate duraton ratio
duration_1q =0.1

In [19]:
from slam.utils.transpiler_pass.speed_limit_pass import \
    pass_manager_basic, \
    pass_manager_optimized_sqiswap
    # pass_manager_slam

""" 
Basic, uses alibaba rules
Optimized, use parallel gate optimized CPhase-family and SWAP ruless
"""
speed_method = 'linear'
pm_basic = pass_manager_basic(gate='sqiswap', duration_1q=duration_1q)
pm_optimized = pass_manager_optimized_sqiswap(duration_1q=duration_1q, speed_method=speed_method)

In [20]:
# first, import the circuit benchmarks
from slam.utils.circuit_suite import benchmark_lambdas

# define the topology
from qiskit import transpile
from qiskit.transpiler import CouplingMap
coupling_map = CouplingMap.from_grid(4,4)
induce_swaps = lambda qc: transpile(qc, coupling_map=coupling_map, optimization_level=3)

# repeat, only keep best
for benchmark_circuit in benchmark_lambdas[0:1]:

    # best, defined by shortest duration
    basic_best = (None, None, None, None)
    optimized_best = (None, None, None, None)

    for _ in data_repitition:
        qc = benchmark_circuit(16) # instantiate of size 16

        # before inducing, turn off logging for readability
        logger.setLevel(logging.ERROR)
        qc = induce_swaps(qc) # transpile to induce swaps

        # second, run the benchmark
        qc_basic = pm_basic.run(qc)
        duration = pm_basic.property_set['duration']
        gate_counts = pm_basic.property_set['gate_counts']
        crit_counts = pm_basic.property_set['longest_path_counts']

        # check if best
        if basic_best[0] is None or duration < basic_best[0]:
            basic_best = (duration, gate_counts, crit_counts, qc_basic)
        
        # repeat on optimized
        qc_optimized = pm_optimized.run(qc)
        duration = pm_optimized.property_set['duration']
        gate_counts = pm_optimized.property_set['gate_counts']
        crit_counts = pm_optimized.property_set['longest_path_counts']

        # check if best
        if optimized_best[0] is None or duration < optimized_best[0]:
            optimized_best = (duration, gate_counts, crit_counts, qc_optimized)
            
    # print results
    print('basic', basic_best[0])
    print('optimized', optimized_best[0])
    assert basic_best[0] >= optimized_best[0]

basic 126.09999999999951
optimized 104.19999999999978


In [21]:
# qc.draw(output='mpl')

In [22]:
# basic_best[3].draw(output='mpl')

In [23]:
# optimized_best[3].draw(output='mpl')

In [24]:
# third, use ret longest path data to convert to fidelity ratios
# TODO